# QSwitch Usage Tutorial:

In this tutorial you will learn how to operate a QSwitch with the help of the provided driver in `qswitch_driver.py`

### 1. QSwitch Overview: 

The QSwitch is a remote-controlled 8 channel breakout box for 24-line Fischer (or Micro-D) connector
terminated cables. The QSwitch is designed for being placed between for example a 24-channel DC
voltage source such as the QDAC-II Compact or a 24-channel manual breakout box such as the QBox
(both from Quantum Machines) and the fridge wiring, so that fully automated breakout of any of the 24
DC/low frequency lines from the fridge – the signal lines - is possible. 

Inside the QSwitch there are 10x24 relays which can be toggled individually. Eight of the relay groups are
for breaking out to the BNC connectors. One group (!0) is for individual (soft) grounding of the device
signal lines. The last group (!9) is for connecting the instrument at the input (IN) to the internal signal
lines. See image below for a schematic view of the QSwitch architecture:

<img src="./qswitch_schematic.png" width="300" height="200">

For more information regarding the Qswitch visit this [webpage](https://qm.quantum-machines.co/87kjeif6). 

### 2. Importing the driver and connecting to the QSwitch:

In [1]:
from qswitch_driver import QSwitch  # The Python class to control the QSwitch
from qswitch_driver import (
    UDPConfig,
)  # For Ethernet communication with UDP protocol, recommended for Firmware 2.0 and above
from qswitch_driver import VISAConfig  # For USB communication with VISA protocol.
from qswitch_driver import find_qswitch_on_usb  # Function to find the QSwitch on USB

In [2]:
connection_type = "ethernet"  # "ethernet" or "usb"

debug = True  # Set to True to print QSwitch state at each cell


def print_qswitch_state(qswitch: QSwitch, debug: bool = True):
    return qswitch.overview() if debug else None

In [3]:
if connection_type == "ethernet":
    # Connection via Ethernet:
    ip_address = "192.168.8.16"
    qswitch = QSwitch(UDPConfig(ip_address))
elif connection_type == "usb":
    # Connection via USB:
    usb_address = find_qswitch_on_usb()
    qswitch = QSwitch(VISAConfig(usb_address))

### 3. Manually opening and closing relays

Let's connect and unground line 1 to have a passthrough the Qswitch. 

In [4]:
# Print the current status of the QSwitch
qswitch.overview()

{'1': ['grounded'],
 '2': ['grounded'],
 '3': ['grounded'],
 '4': ['grounded'],
 '5': ['grounded'],
 '6': ['grounded'],
 '7': ['grounded'],
 '8': ['grounded'],
 '9': ['grounded'],
 '10': ['grounded'],
 '11': ['grounded'],
 '12': ['grounded'],
 '13': ['grounded'],
 '14': ['grounded'],
 '15': ['grounded'],
 '16': ['grounded'],
 '17': ['grounded'],
 '18': ['grounded'],
 '19': ['grounded'],
 '20': ['grounded'],
 '21': ['grounded'],
 '22': ['grounded'],
 '23': ['grounded'],
 '24': ['grounded']}

In [7]:
#################################################################################################################
# NOTE: Line and tap correspond to the horizontal and vertical axis of the QSwitch matrix in the image above.   #
#################################################################################################################

# First connect line 1 by closing it's input relay on tap 9.
qswitch.close_relay(line=1, tap=9)
# Now unground line 1 by opening the output relay on tap 0.
qswitch.open_relay(line=1, tap=0)

print_qswitch_state(qswitch, debug)

{'1': ['connected'],
 '2': ['grounded'],
 '3': ['grounded'],
 '4': ['grounded'],
 '5': ['grounded'],
 '6': ['grounded'],
 '7': ['grounded'],
 '8': ['grounded'],
 '9': ['grounded'],
 '10': ['grounded'],
 '11': ['grounded'],
 '12': ['grounded'],
 '13': ['grounded'],
 '14': ['grounded'],
 '15': ['grounded'],
 '16': ['grounded'],
 '17': ['grounded'],
 '18': ['grounded'],
 '19': ['grounded'],
 '20': ['grounded'],
 '21': ['grounded'],
 '22': ['grounded'],
 '23': ['grounded'],
 '24': ['grounded']}

In [ ]:
# We can now connect it line one to BNC breakout number 1 like this:
qswitch.close_relay(line=1, tap=1)

print_qswitch_state(qswitch, debug)

In [ ]:
# Open and close multiple relays at once:
qswitch.open_relays(relays=[(2, 0), (3, 0), (4, 0), (5, 0)])
qswitch.close_relays(relays=[(2, 9), (3, 9), (4, 9), (5, 9)])

print_qswitch_state(qswitch, debug)

### 4. Multi-Operation commands

Operations across multiple QSwitch line and taps are pre-coded in the QSwitch class. 
Operations are executed in the specific order indicated by the operation name.

In [ ]:
# Connecting and ungrounding a line:
qswitch.connect_and_unground(lines="6")
# Connecting and ungrounding multiple lines:
qswitch.connect_and_unground(lines=["7", "8", "9", "10"])

print_qswitch_state(qswitch, debug)

In [ ]:
# Releasing and grounding a line:
qswitch.ground_and_release(lines="6")
# Releasing and frounding multiple lines:
qswitch.ground_and_release(lines=["7", "8", "9", "10"])

print_qswitch_state(qswitch, debug)

In [ ]:
# Connecting a line to a BNC breakout:
qswitch.breakout(line="11", tap="2")  # Witch is equivalent to qswitch.close_relay(8, 2) and qswitch.open_relay(8, 0)
# NOTE: This function will not connect the input line (tap 9)

print_qswitch_state(qswitch, debug)

### 5. Connecting and disconnecting all lines at once

In [ ]:
# Ground and release all lines: (default QSwitch state)
qswitch.ground_and_release_all()  # Will also disconnect all BNC breakouts

print_qswitch_state(qswitch, debug)

In [ ]:
# Connect and unground all lines:
qswitch.connect_and_unground_all()  # Will not affect the BNC breakouts

print_qswitch_state(qswitch, debug)

### 6. Renaming lines and taps:

In [ ]:
# You can rename the lines and breakouts to custom names instead of numbers with the arrange function:
qswitch.arrange(
    lines={
        "Plunger_1": 1,
        "Plunger_2": 2,
        "Barrier_1": 23,
        "Barrier_2": 24,
    },
    breakouts={
        "GROUND": 0,
        "DMM": 1,
        "IV_CONVERTER": 2,
        "CONNECTOR": 9,
    },
)
# Reset the QSwitch to default state:
qswitch.ground_and_release_all()
# Now you can use the custom names instead of numbers:
qswitch.connect_and_unground(lines="Plunger_1")
qswitch.breakout(line="Barrier_2", tap="DMM")

print_qswitch_state(qswitch, debug)

### 7. Autosave mode:

When the QSwitch is unpowered, each signal line is connected to soft ground (through 1 MΩ) and
disconnected from the input and all BNC breakout lines, so that it is safe to connect a device to the
output (OUT).

In case of power loss, the relays connecting the signal lines to soft ground will close within
approximately 2 milliseconds whereafter the relays connecting to the input side and to the breakouts
will open within 200 ms in random order.

Autosave mode will automatically re-apply the state of the QSwitch before Power-Loss. To activate autosave use:

```python
qswitch.auto_save("on")
```

### Close the connection to the QSwitch:

In [ ]:
qswitch.ground_and_release_all()
qswitch.close()  # Close the connection to the QSwitch, important when using Ethernet connection to not run out of connections